In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Portobelo,PA,2022-11-08 16:15:35,9.5500,-79.6500,78.12,85,100,7.00
1,1,Dalbandin,PK,2022-11-08 16:15:36,28.8947,64.4101,68.00,44,45,7.47
2,2,Port Lincoln,AU,2022-11-08 16:15:36,-34.7333,135.8667,71.74,53,85,19.75
3,3,Saldanha,ZA,2022-11-08 16:15:36,-33.0117,17.9442,62.51,72,94,14.88
4,4,Seymchan,RU,2022-11-08 16:15:37,62.8833,152.4333,-6.27,91,100,6.02


In [3]:
#Get the data types
city_data_df.dtypes


City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [15]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Portobelo,PA,2022-11-08 16:15:35,9.5500,-79.6500,78.12,85,100,7.00
7,7,Vaini,TO,2022-11-08 16:15:38,-21.2000,-175.2000,75.36,73,75,8.05
16,16,Road Town,VG,2022-11-08 16:13:42,18.4167,-64.6167,84.29,84,20,12.66
17,17,Kapaa,US,2022-11-08 16:15:41,22.0752,-159.3190,77.09,84,75,12.66
19,19,Rock Sound,BS,2022-11-08 16:11:04,24.9000,-76.2000,79.20,77,52,25.52
21,21,Hithadhoo,MV,2022-11-08 16:15:42,-0.6000,73.0833,81.41,79,83,17.13
23,23,Sur,OM,2022-11-08 16:15:42,22.5667,59.5289,83.55,56,71,6.08
24,24,Butaritari,KI,2022-11-08 16:15:22,3.0707,172.7902,80.29,74,98,10.33
32,32,Dutlwe,BW,2022-11-08 16:15:45,-23.9833,23.9000,79.39,32,1,12.19
36,36,Bengkulu,ID,2022-11-08 16:15:46,-3.8004,102.2655,76.84,90,100,4.74


In [21]:
preferred_cities_df.count()

City_ID       12
City          12
Country       12
Date          12
Lat           12
Lng           12
Max Temp      12
Humidity      12
Cloudiness    12
Wind Speed    12
dtype: int64

In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Portobelo,PA,78.12,9.5500,-79.6500,
7,Vaini,TO,75.36,-21.2000,-175.2000,
16,Road Town,VG,84.29,18.4167,-64.6167,
17,Kapaa,US,77.09,22.0752,-159.3190,
19,Rock Sound,BS,79.20,24.9000,-76.2000,
21,Hithadhoo,MV,81.41,-0.6000,73.0833,
23,Sur,OM,83.55,22.5667,59.5289,
24,Butaritari,KI,80.29,3.0707,172.7902,
32,Dutlwe,BW,79.39,-23.9833,23.9000,
36,Bengkulu,ID,76.84,-3.8004,102.2655,


In [23]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [35]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Portobelo,PA,78.12,9.5500,-79.6500,Scuba Portobelo
7,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
16,Road Town,VG,84.29,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
17,Kapaa,US,77.09,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Rock Sound,BS,79.20,24.9000,-76.2000,Pink Corral Bay Property
21,Hithadhoo,MV,81.41,-0.6000,73.0833,Scoop Guest House
23,Sur,OM,83.55,22.5667,59.5289,Sur Plaza Hotel
24,Butaritari,KI,80.29,3.0707,172.7902,Isles Sunset Lodge
32,Dutlwe,BW,79.39,-23.9833,23.9000,
36,Bengkulu,ID,76.84,-3.8004,102.2655,Grage Hotel Bengkulu


In [36]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [45]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))